In [1]:
import numpy as np
import pandas as pd
df_med = pd.read_csv("./data_wrangling_medical_2021_u6773547.csv", dtype = {'postcode':str})
df_edu = pd.read_csv("./data_wrangling_education_2021_u6773547.csv", dtype = {'postcode':str})

In [2]:
print(df_med.postcode[0])

0828


In [3]:
# 发现问题

# 1.检查len
print(len(df_med.ssn))
print(len(df_edu.ssn))

# 2.检查unique
df_med_unique = np.unique(df_med.ssn)
df_edu_unique = np.unique(df_edu.ssn)
print(df_med_unique.size)
print(df_edu_unique.size)

# 3.检查nan
df_med_nan = df_med.ssn.isna().sum()
df_edu_nan = df_edu.ssn.isna().sum()
print(df_med_nan)
print(df_edu_nan)

# 4.how many same values of ssn in x and y
sw = [x for x in df_med_unique if x in df_edu_unique]
print(len(sw)) # 方法一
print(len(set(df_med.ssn) & set(df_edu.ssn))) # 方法二(输出相同元素)
# 说明df_edu的ssn有重复值

20000
20000
20000
19215
0
0
15991
15991


In [4]:
print(len(df_med.ssn))

20000


In [5]:
# 数据预处理
# 1.df_edu中的ssn有重复项
df_edu.drop_duplicates('ssn','first',inplace = True)
print(len(df_edu.ssn))

19215


In [6]:
# Q2.1 merge data sets
df_merge = pd.merge(df_med, df_edu, how='inner', on='ssn')
# 20000(df_med.ssn) + 19215(df_edu.ssn) - 15991(same values) = 23224
print(len(df_merge.ssn))

15991


In [7]:
df_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15991 entries, 0 to 15990
Data columns (total 43 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   rec_id_x                15991 non-null  object 
 1   ssn                     15991 non-null  object 
 2   first_name_x            15991 non-null  object 
 3   middle_name_x           14385 non-null  object 
 4   last_name_x             15991 non-null  object 
 5   gender_x                15991 non-null  object 
 6   age_at_consultation     15991 non-null  int64  
 7   birth_date_x            15991 non-null  object 
 8   medicare_number         15991 non-null  object 
 9   street_address_x        15991 non-null  object 
 10  suburb_x                15991 non-null  object 
 11  postcode_x              12803 non-null  object 
 12  state_x                 15991 non-null  object 
 13  phone_x                 9527 non-null   object 
 14  email_x                 10974 non-null

In [16]:
j = 0
for i in range (0, len(df_merge.gender_x)):
    if df_merge.gender_x[i] != df_merge.gender_y[i]:
        j += 1
print(j)

1624


In [8]:
# Q2.4
# 把两个dataset都有的列打印出来
list_common = ["rec_id","first_name","middle_name","last_name","gender","birth_date","street_address","suburb","postcode","state","phone","email"]
x = []
y = []

def print_list(list_x,list_y):
    for i in list_common:    
        list_x = (i + "_x")
        x.append(list_x)
        list_y = (i + "_y")
        y.append(list_y)
    return x,y


In [9]:
print(df_merge.postcode_x)

0        0828
1         NaN
2        2444
3        6280
4        2539
         ... 
15986     NaN
15987    3862
15988     NaN
15989    4350
15990     NaN
Name: postcode_x, Length: 15991, dtype: object


In [10]:
# 寻找两个data sets中共同列的不一致的keys，用list保存起来
# 手工算13组数据
incon_list = []
def fin_incon():        
    for i in range (0, len(df_merge.gender_x)):
        if df_merge.gender_x[i] != df_merge.gender_y[i]:
            incon_list.append(i)
    return len(incon_list)
print(fin_incon())

1624


In [11]:
# consultation_timestamp x mde
# employment_timestamp y edu
def two_choose_one():
    if df_merge['consultation_timestamp'][15986] >= df_merge['employment_timestamp'][15986]:
        return df_merge['consultation_timestamp'][15986]    
    else: 
        return df_merge['employment_timestamp'][15986]
two_choose_one()

'2020-05-23t17:11+00:00'

In [12]:
# 两两对比，添加到df_merge上面就行
data = {'first_name_x':df_merge['first_name_x'], 'first_name_y':df_merge['first_name_y'], \
        'middle_name_x':df_merge['middle_name_x'], 'middle_name_y':df_merge['middle_name_y'],\
        'last_name_x':df_merge['last_name_x'], 'last_name_y':df_merge['last_name_y'],\
        'gender_x':df_merge['gender_x'], 'gender_y':df_merge['gender_y'],\
        'birth_date_x':df_merge['birth_date_x'], 'birth_date_y':df_merge['birth_date_y'],\
        'street_address_x':df_merge['street_address_x'], 'street_address_y':df_merge['street_address_y'],\
        'suburb_x':df_merge['suburb_x'], 'suburb_y':df_merge['suburb_y'],\
        'postcode_x':df_merge['postcode_x'], 'postcode_y':df_merge['postcode_y'],\
        'state_x':df_merge['state_x'], 'state_y':df_merge['state_y'],\
        'phone_x':df_merge['phone_x'], 'phone_y':df_merge['phone_y'],\
        'email_x':df_merge['email_x'], 'email_y':df_merge['email_y'],\
        'postcode_x':df_merge['postcode_x'], 'postcode_y':df_merge['postcode_y'],\
        'consultation_timestamp':df_merge['consultation_timestamp'],'employment_timestamp':df_merge['employment_timestamp'], \
           }
df = pd.DataFrame(data)
df

,first_name_x,first_name_y,middle_name_x,middle_name_y,last_name_x,last_name_y,gender_x,gender_y,birth_date_x,birth_date_y,...,postcode_x,postcode_y,state_x,state_y,phone_x,phone_y,email_x,email_y,consultation_timestamp,employment_timestamp
0,joseph,joseph,jackson,jackson,rothstein,rothstein,m,m,16/6/2005,16/6/2005,...,0828,7300,nt,tas,NaN,03 6131 4813,nmbhiabbfr.yahoo.com,nmbhiabbfr@yahoo.com,2015-06-02t10:28+00:00,2020-07-07t23:37+00:00
1,james,james,m,m,hinnant,hinnant,f,f,9/9/1955,9/9/1955,...,NaN,2025,vic,nsw,NaN,NaN,james44.yahoo.com,james44@yahoo.com,2016-09-12t16:21+00:00,2020-02-22t02:40+00:00
2,michael,michael,juan,juan,whitfield,whitfield,f,f,10/12/2005,10/12/2005,...,2444,2444,nsw,nsw,02 7021 1718,02 7021 1718,michael49@mail.com.au,michael49@mail.com.au,2020-03-03t12:35+00:00,2020-07-07t16:11+00:00
3,bryan,bryan,lawrence,lawrence,roos,roos,f,f,15/10/2003,15/10/2003,...,6280,3184,wa,vic,NaN,03 9318 7858,pcddkhdsnp@mail.com,NaN,2016-12-06t23:57+00:00,2020-06-02t23:26+00:00
4,william,william,earl,earl,galyean,galyean,f,f,2/3/1965,2/3/1965,...,2539,2539,nsw,nsw,02 2894 1600,02 2894 1600,galyean1@hotmail.com,galyean1@hotmail.com,2014-12-01t15:34+00:00,2017-05-02t03:38+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15986,louise,louise,mae,mae,brown,brown,f,f,26/1/1997,26/1/1997,...,NaN,7008,tas,tas,03 5484 1708,03 5484 1708,brown.louise@mail.com,brown.louise@mail.com,2013-12-21t11:38+00:00,2020-05-23t17:11+00:00
15987,reginald,reginald,thomas,thomas,ballentine,ballentine,m,m,2/4/1957,2/4/1957,...,3862,3862,vic,vic,NaN,03 4333 2710,reginald16@mail.com,reginald16@mail.com,2013-09-26t10:57+00:00,2019-03-03t15:35+00:00
15988,victoria,victoria,diane,diane,goodman,goodman,f,f,16/6/1987,16/6/1987,...,NaN,3152,vic,vic,NaN,03 2137 5224,victoria12@gmail.com,victoria12@gmail.com,2013-11-10t18:09+00:00,2011-05-10t23:21+00:00
15989,kisha,kisha,michael,michael,depaulo,depaulo,m,m,27/9/2003,27/9/2003,...,4350,4350,qld,qld,07 8212 7771,07 8212 7771,NaN,depaulo83@hotmail.com,2016-01-15t08:38+00:00,2020-06-13t02:55+00:00


In [13]:
df_merge['first_name'] = df.apply(lambda x: x.first_name_x \
                            if x.consultation_timestamp >= x.employment_timestamp \
                            else x.first_name_y, axis=1)
df_merge['middle_name'] = df.apply(lambda x: x.middle_name_x \
                            if x.consultation_timestamp >= x.employment_timestamp \
                            else x.middle_name_y, axis=1)
df_merge['last_name'] = df.apply(lambda x: x.last_name_x \
                            if x.consultation_timestamp >= x.employment_timestamp \
                            else x.last_name_y, axis=1)
df_merge['gender'] = df.apply(lambda x: x.gender_x \
                            if x.consultation_timestamp >= x.employment_timestamp \
                            else x.gender_y, axis=1)
df_merge['birth_date'] = df.apply(lambda x: x.birth_date_x \
                            if x.consultation_timestamp >= x.employment_timestamp \
                            else x.birth_date_y, axis=1)
df_merge['street_address'] = df.apply(lambda x: x.street_address_x \
                            if x.consultation_timestamp >= x.employment_timestamp \
                            else x.street_address_y, axis=1)
df_merge['suburb'] = df.apply(lambda x: x.suburb_x \
                            if x.consultation_timestamp >= x.employment_timestamp \
                            else x.suburb_y, axis=1)
df_merge['postcode'] = df.apply(lambda x: x.postcode_x \
                            if x.consultation_timestamp >= x.employment_timestamp \
                            else x.postcode_y, axis=1)
df_merge['state'] = df.apply(lambda x: x.state_x \
                            if x.consultation_timestamp >= x.employment_timestamp \
                            else x.state_y, axis=1)
df_merge['phone'] = df.apply(lambda x: x.phone_x \
                            if x.consultation_timestamp >= x.employment_timestamp \
                            else x.phone_y, axis=1)
df_merge['email'] = df.apply(lambda x: x.email_x \
                            if x.consultation_timestamp >= x.employment_timestamp \
                            else x.email_y, axis=1)
df_merge['postcode'] = df.apply(lambda x: x.postcode_x \
                            if x.consultation_timestamp >= x.employment_timestamp \
                            else x.postcode_y, axis=1)

df_merge

,rec_id_x,ssn,first_name_x,middle_name_x,last_name_x,gender_x,age_at_consultation,birth_date_x,medicare_number,street_address_x,...,middle_name,last_name,gender,birth_date,street_address,suburb,postcode,state,phone,email
0,rec-38255,a119167529,joseph,jackson,rothstein,m,9,16/6/2005,1600 34634 2 3,1 lagoon road house 106,...,jackson,rothstein,m,16/6/2005,21 drummond crescent glenrowan hmes,perth,7300,tas,03 6131 4813,nmbhiabbfr@yahoo.com
1,rec-96960,h148559060,james,m,hinnant,f,60,9/9/1955,8787 78232 1 2,224 grove road site shed deakin,...,m,hinnant,f,9/9/1955,64 ocean street pond house,woollahra,2025,nsw,NaN,james44@yahoo.com
2,rec-89846,i182575302,michael,juan,whitfield,f,15,10/12/2005,8938 59176 1 2,4 surf street greysands,...,juan,whitfield,f,10/12/2005,4 surf street greysands,flynns beach,2444,nsw,02 7021 1718,michael49@mail.com.au
3,rec-85752,e141244372,bryan,lawrence,roos,f,13,15/10/2003,2655 78596 2 2,20 ray avenue rosa brook wing,...,lawrence,roos,f,15/10/2003,389 barkly street flt 3,elwood,3184,vic,03 9318 7858,NaN
4,rec-82035,b165896593,william,earl,galyean,f,49,2/3/1965,3520 26642 2 2,25 princes highway conjola mountain,...,earl,galyean,f,2/3/1965,25 princes highway conjola mountain,conjola,2539,nsw,02 2894 1600,galyean1@hotmail.com
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15986,rec-11186,g171813397,louise,mae,brown,f,16,26/1/1997,2444 21135 1 3,225 cadbury road derwent waters,...,mae,brown,f,26/1/1997,49 augusta road calvary hospital marian ...,lenah valley,7008,tas,03 5484 1708,brown.louise@mail.com
15987,rec-65265,f140131186,reginald,thomas,ballentine,m,56,2/4/1957,6621 46650 1 2,75 lathams road stedmoor,...,thomas,ballentine,m,2/4/1957,75 lathams road stedmoor,meerlieu,3862,vic,03 4333 2710,reginald16@mail.com
15988,rec-53589,g150433327,victoria,diane,goodman,f,26,16/6/1987,7936 65928 1 1,100 harold street salford park,...,diane,goodman,f,16/6/1987,100 harold street salford park,wantirna,NaN,vic,NaN,victoria12@gmail.com
15989,rec-2147,a174082079,kisha,michael,depaulo,m,13,27/9/2003,1996 11945 2 2,258 spring street yukana retirement vlge,...,michael,depaulo,m,27/9/2003,258 spring street yukana retirement vlge,toowoomba,4350,qld,07 8212 7771,depaulo83@hotmail.com


In [14]:
# 删除x y的列，除了rec_id
# Q2完结
df_new_data = df_merge.drop(['first_name_x', 'middle_name_x', 'last_name_x', 'gender_x', 'birth_date_x', 'street_address_x', 'suburb_x', 'postcode_x', 'state_x', 'phone_x', 'email_x',\
              'first_name_y', 'middle_name_y', 'last_name_y', 'gender_y', 'birth_date_y', 'street_address_y', 'suburb_y', 'postcode_y', 'state_y', 'phone_y', 'email_y'],axis=1)
df_new_data

,rec_id_x,ssn,age_at_consultation,medicare_number,marital_status,height,weight,bmi,blood_pressure,cholesterol_level,...,middle_name,last_name,gender,birth_date,street_address,suburb,postcode,state,phone,email
0,rec-38255,a119167529,9,1600 34634 2 3,NaN,136,52,28,69,143,...,jackson,rothstein,m,16/6/2005,21 drummond crescent glenrowan hmes,perth,7300,tas,03 6131 4813,nmbhiabbfr@yahoo.com
1,rec-96960,h148559060,60,8787 78232 1 2,married,174,65,21,77,203,...,m,hinnant,f,9/9/1955,64 ocean street pond house,woollahra,2025,nsw,NaN,james44@yahoo.com
2,rec-89846,i182575302,15,8938 59176 1 2,NaN,165,88,32,78,116,...,juan,whitfield,f,10/12/2005,4 surf street greysands,flynns beach,2444,nsw,02 7021 1718,michael49@mail.com.au
3,rec-85752,e141244372,13,2655 78596 2 2,NaN,148,68,31,82,186,...,lawrence,roos,f,15/10/2003,389 barkly street flt 3,elwood,3184,vic,03 9318 7858,NaN
4,rec-82035,b165896593,49,3520 26642 2 2,married-de-facto,185,-99,40,80,231,...,earl,galyean,f,2/3/1965,25 princes highway conjola mountain,conjola,2539,nsw,02 2894 1600,galyean1@hotmail.com
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15986,rec-11186,g171813397,16,2444 21135 1 3,married,179,77,24,71,164,...,mae,brown,f,26/1/1997,49 augusta road calvary hospital marian ...,lenah valley,7008,tas,03 5484 1708,brown.louise@mail.com
15987,rec-65265,f140131186,56,6621 46650 1 2,widowed,194,56,14,80,304,...,thomas,ballentine,m,2/4/1957,75 lathams road stedmoor,meerlieu,3862,vic,03 4333 2710,reginald16@mail.com
15988,rec-53589,g150433327,26,7936 65928 1 1,married,172,92,31,72,155,...,diane,goodman,f,16/6/1987,100 harold street salford park,wantirna,NaN,vic,NaN,victoria12@gmail.com
15989,rec-2147,a174082079,13,1996 11945 2 2,NaN,140,40,20,76,146,...,michael,depaulo,m,27/9/2003,258 spring street yukana retirement vlge,toowoomba,4350,qld,07 8212 7771,depaulo83@hotmail.com


In [15]:
df_new_data.to_csv("new_data.csv", index = False)